In [4]:
from cassandra.cluster import Cluster, PlainTextAuthProvider
from cassandra.cluster import (
    NoHostAvailable,
    OperationTimedOut
)
import os,logging,sys,uuid


ToDo: 

* Rebuild Database with updated data versions

* Update code bases

    * protobufs

    * worker(s)

* Solve for the TLS issue

* 

In [5]:
""" 
SELECT * FROM characters WHERE campaign_id = 'DNDIO' and user_id = 'Chorky#8402;
"""
CASS_USER = os.getenv('CASS_USER') or 'cassandra'
CASS_PASS = os.getenv('CASS_PASS') or 'changeme'
CASS_DB = os.getenv('CASS_DB') or 'dndio'
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[handler]
)
logger = logging.getLogger(__name__)

In [ ]:
class cassDB():
    def __init__(self,user,pwd,host,db):
        self.auth_provider = PlainTextAuthProvider(
            username=user,
            password = pwd
        )
        self.host = host
        self.db = db
        self.cluster= None 
        self.session = None
        # self.connect()
    def connect(self):
        logger.info( " [x] Initializating DB Connection to Cassandra @ {}".format(self.host))
        self.cluster = Cluster(
            [self.host],
            auth_provider=self.auth_provider
        )
        logger.info("  [x] Establishing cluster session")
        self.session = self.cluster.connect()
        logger.info("  [x] Connecting to database: {}".format(self.db))
        self.session.set_keyspace(self.db)
        self.session.execute('USE {};'.format(self.db))
        logger.info( " [x] Connected to Cassandra Database!")

    def rows_to_json(self,rows):
        result = []
        for row in rows:
            d = row._asdict()
            for k,v in d.items():
                if isinstance(v,uuid.UUID):
                    d[k] = str(v)
            result.append(d)
        return result

    def exec_query(self,s):
        result = {
            'query':s,
            'success':False
        }
        try: 
            response = self.session.execute(s)
        except NoHostAvailable:
            self.connect()
        except OperationTimedOut:
            self.connect()
        finally:
            response = self.session.execute(s)
            result['success']=True
        logger.info("  [x] DB on_request: query succeeded: {}".format(s))
        # l = [r for r in response.all()]
        # return response.all()
        result['rows'] = self.rows_to_json(response.all())
        return result

In [91]:
db = cassDB(CASS_USER,CASS_PASS,'localhost',CASS_DB)

In [92]:
db.connect()

2024-11-22 08:09:09,708 - __main__ - INFO -  [x] Initializating DB Connection to Cassandra @ localhost
2024-11-22 08:09:09,710 - cassandra.cluster - WARNING - Cluster.__init__ called with contact_points specified, but no load_balancing_policy. In the next major version, this will raise an error; please specify a load-balancing policy. (contact_points = ['localhost'], lbp = None)
2024-11-22 08:09:09,713 - __main__ - INFO -   [x] Establishing cluster session
2024-11-22 08:09:10,466 - cassandra.cluster - WARNING - Downgrading core protocol version from 66 to 65 for 127.0.0.1:9042. To avoid this, it is best practice to explicitly set Cluster(protocol_version) to the version supported by your cluster. http://datastax.github.io/python-driver/api/cassandra/cluster.html#cassandra.cluster.Cluster.protocol_version
2024-11-22 08:09:11,989 - cassandra.cluster - WARNING - Downgrading core protocol version from 65 to 5 for 127.0.0.1:9042. To avoid this, it is best practice to explicitly set Cluster(

In [ ]:
res =db.exec_query(
    "SELECT * FROM characters WHERE campaign_id = 'abcdef' and user_id='chorky#8402'"
)
res

2024-11-22 08:09:15,931 - cassandra.query - WARNING - Failed creating named tuple for results with column names ['campaign_id', 'user_id', 'ac', 'align', 'background', 'char_name', 'chr', 'class', 'con', 'curr_hp', 'dex', 'hit_dice', 'int', 'level', 'max_hp', 'prof_bonus', 'str', 'wis'] (cleaned: ['campaign_id', 'user_id', 'ac', 'align', 'background', 'char_name', 'chr', 'class', 'con', 'curr_hp', 'dex', 'hit_dice', 'int', 'level', 'max_hp', 'prof_bonus', 'str', 'wis']) (see Python 'namedtuple' documentation for details on name rules). Results will be returned with positional names. Avoid this by choosing different names, using SELECT "<col name>" AS aliases, or specifying a different row_factory on your Session
2024-11-22 08:09:15,946 - cassandra.query - WARNING - Failed creating named tuple for results with column names ['campaign_id', 'user_id', 'ac', 'align', 'background', 'char_name', 'chr', 'class', 'con', 'curr_hp', 'dex', 'hit_dice', 'int', 'level', 'max_hp', 'prof_bonus', 'str

{'query': "SELECT * FROM characters WHERE campaign_id = 'abcdef' and user_id='chorky#8402'",
 'success': False,
 'rows': [{'campaign_id': 'abcdef',
   'user_id': 'chorky#8402',
   'ac': 18,
   'align': None,
   'background': None,
   'char_name': 'Bahlok',
   'chr': 20,
   'field_7_': None,
   'con': None,
   'curr_hp': None,
   'dex': None,
   'hit_dice': None,
   'int': None,
   'level': None,
   'max_hp': None,
   'prof_bonus': None,
   'str': None,
   'wis': None}]}

2024-11-22 08:09:18,761 - cassandra.cluster - WARNING - Failed to create connection pool for new host 10.1.2.159:9042:
Traceback (most recent call last):
  File "c:\Users\pconn\miniconda3\lib\site-packages\cassandra\cluster.py", line 3234, in run_add_or_renew_pool
    new_pool = HostConnection(host, distance, self)
  File "c:\Users\pconn\miniconda3\lib\site-packages\cassandra\pool.py", line 406, in __init__
    self._connection = session.cluster.connection_factory(host.endpoint, on_orphaned_stream_released=self.on_orphaned_stream_released)
  File "c:\Users\pconn\miniconda3\lib\site-packages\cassandra\cluster.py", line 1670, in connection_factory
    return self.connection_class.factory(endpoint, self.connect_timeout, *args, **kwargs)
  File "c:\Users\pconn\miniconda3\lib\site-packages\cassandra\connection.py", line 846, in factory
    conn = cls(endpoint, *args, **kwargs)
  File "c:\Users\pconn\miniconda3\lib\site-packages\cassandra\io\asyncorereactor.py", line 353, in __init__
    sel

In [71]:
for row in res:
    d = row._asdict()
    for k,v in d.items():
        if isinstance(v,uuid.UUID):
            d[k] = str(v)
    print(d)

{'campaign_id': 'abcdef', 'user_id': 'chorky#8402', 'ac': 18, 'align': None, 'background': None, 'char_name': 'Bahlok', 'chr': 20, 'field_7_': None, 'con': None, 'curr_hp': None, 'dex': None, 'hit_dice': None, 'int': None, 'level': None, 'max_hp': None, 'prof_bonus': None, 'str': None, 'wis': None}


2024-11-22 08:04:08,137 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.166:9042, scheduling retry in 276.48 seconds: [Errno None] Tried connecting to [('10.1.2.166', 9042)]. Last error: timed out


In [75]:
len(res)

1

2024-11-22 08:05:06,233 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.159:9042, scheduling retry in 108.8 seconds: [Errno None] Tried connecting to [('10.1.2.159', 9042)]. Last error: timed out
2024-11-22 08:05:07,356 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.166:9042, scheduling retry in 445.44 seconds: [Errno None] Tried connecting to [('10.1.2.166', 9042)]. Last error: timed out
2024-11-22 08:05:15,507 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.166:9042, scheduling retry in 144.64 seconds: [Errno None] Tried connecting to [('10.1.2.166', 9042)]. Last error: timed out


In [94]:
res

{'query': "SELECT * FROM characters WHERE campaign_id = 'abcdef' and user_id='chorky#8402'",
 'success': False,
 'rows': [{'campaign_id': 'abcdef',
   'user_id': 'chorky#8402',
   'ac': 18,
   'align': None,
   'background': None,
   'char_name': 'Bahlok',
   'chr': 20,
   'field_7_': None,
   'con': None,
   'curr_hp': None,
   'dex': None,
   'hit_dice': None,
   'int': None,
   'level': None,
   'max_hp': None,
   'prof_bonus': None,
   'str': None,
   'wis': None}]}

In [97]:
char = res['rows'][0]

In [98]:
char

{'campaign_id': 'abcdef',
 'user_id': 'chorky#8402',
 'ac': 18,
 'align': None,
 'background': None,
 'char_name': 'Bahlok',
 'chr': 20,
 'field_7_': None,
 'con': None,
 'curr_hp': None,
 'dex': None,
 'hit_dice': None,
 'int': None,
 'level': None,
 'max_hp': None,
 'prof_bonus': None,
 'str': None,
 'wis': None}

In [107]:
db.exec_query("UPDATE characters SET str=18,wis=9,con=12,dex=15,int=11,level=6 WHERE campaign_id='abcdef' and user_id='chorky#8402';")

2024-11-22 08:13:13,511 - __main__ - INFO -   [x] DB on_request: query succeeded: UPDATE characters SET str=18,wis=9,con=12,dex=15,int=11,level=6 WHERE campaign_id='abcdef' and user_id='chorky#8402';


{'query': "UPDATE characters SET str=18,wis=9,con=12,dex=15,int=11,level=6 WHERE campaign_id='abcdef' and user_id='chorky#8402';",
 'success': False,
 'rows': []}

In [103]:
db.exec_query("UPDATE characters SET hit_dice={5:8} WHERE campaign_id='abcdef' and user_id='chorky#8402';")

2024-11-22 08:12:23,979 - __main__ - INFO -   [x] DB on_request: query succeeded: UPDATE characters SET hit_dice={5:8} WHERE campaign_id='abcdef' and user_id='chorky#8402';


{'query': "UPDATE characters SET hit_dice={5:8} WHERE campaign_id='abcdef' and user_id='chorky#8402';",
 'success': False,
 'rows': []}

In [108]:
db.exec_query(
    "UPDATE characters SET curr_hp = 42 WHERE campaign_id='abcdef' AND user_id='chorky#8402';"
)

2024-11-22 08:14:45,117 - __main__ - INFO -   [x] DB on_request: query succeeded: UPDATE characters SET curr_hp = 42 WHERE campaign_id='abcdef' AND user_id='chorky#8402';


{'query': "UPDATE characters SET curr_hp = 42 WHERE campaign_id='abcdef' AND user_id='chorky#8402';",
 'success': False,
 'rows': []}

In [111]:
spells = db.exec_query(
    "select spells from char_map WHERE campaign_id='abcdef' AND char_id='chorky#8402';"
)

2024-11-22 08:17:26,992 - __main__ - INFO -   [x] DB on_request: query succeeded: select spells from char_map WHERE campaign_id='abcdef' AND char_id='chorky#8402';


In [127]:
spells['rows'][0]['spells'][0].__str__()

'769a39bf-75ac-4eed-85a7-429d34c2e705'

2024-11-22 08:20:00,431 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.166:9042, scheduling retry in 600.0 seconds: [Errno None] Tried connecting to [('10.1.2.166', 9042)]. Last error: timed out


In [119]:
spell = db.exec_query(
    "select * from spells WHERE name='Eldritch Blast' allow filtering;"
)['rows']

2024-11-22 08:19:02,005 - __main__ - INFO -   [x] DB on_request: query succeeded: select * from spells WHERE name='Eldritch Blast' allow filtering;


In [120]:
spell

[{'id': 'ea42031d-4ab2-4d4a-bbb8-f6c0c80642bf',
  'addspellmod': True,
  'classes': ['Warlock'],
  'components': 'V, S',
  'conc': False,
  'dmg': OrderedMapSerializedKey([(1, OrderedMapSerializedKey([('Force', [OrderedMapSerializedKey([(1, 10)])])]))]),
  'duration': 'Instantaneous',
  'hard_mod': 0,
  'lvl': 0,
  'mult': 1,
  'name': 'Eldritch Blast',
  'range': '120 feet',
  'save': 'nan',
  'time': '1 action',
  'upcast': OrderedMapSerializedKey([])}]

2024-11-22 08:19:09,475 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.159:9042, scheduling retry in 600.0 seconds: [Errno None] Tried connecting to [('10.1.2.159', 9042)]. Last error: timed out


so for spell damage...
have to iterate through the dmg field to convert back to a dictionary instead of a cassandra type
d = {}
for row in result:
    dmg_dict = {}
    dmg = row['dmg']
    for level, dmgmap in dmg.items():
        dmg_dict[level] = {}
        for dmgtype,die_list in dmgmap.items():
            dmg_dict[level][dmgtype] = []
            for die in die_list:
                dmg_dict[level][dmg_type].append(die.keys()[0],die.values()[0])



In [128]:
db.exec_query(
    """ SELECT * FROM spells WHERE id={}""".format(spells['rows'][0]['spells'][0])
)

2024-11-22 08:20:42,634 - __main__ - INFO -   [x] DB on_request: query succeeded:  SELECT * FROM spells WHERE id=769a39bf-75ac-4eed-85a7-429d34c2e705


{'query': ' SELECT * FROM spells WHERE id=769a39bf-75ac-4eed-85a7-429d34c2e705',
 'success': False,
 'rows': []}

In [130]:
r=db.exec_query(
    """ SELECT * FROM spells WHERE id={}""".format(uuid.UUID('ea42031d-4ab2-4d4a-bbb8-f6c0c80642bf'))
)

2024-11-22 08:21:30,073 - __main__ - INFO -   [x] DB on_request: query succeeded:  SELECT * FROM spells WHERE id=ea42031d-4ab2-4d4a-bbb8-f6c0c80642bf


In [134]:
dmg = r['rows'][0]['dmg']

In [135]:
dmg

OrderedMapSerializedKey([(1, OrderedMapSerializedKey([('Force', [OrderedMapSerializedKey([(1, 10)])])]))])

In [137]:
dmg.keys()

KeysView(OrderedMapSerializedKey([(1, OrderedMapSerializedKey([('Force', [OrderedMapSerializedKey([(1, 10)])])]))]))

In [138]:
for k in dmg.keys():
    print(k)

1


In [145]:
tlvl = 18



In [140]:
tlvl

In [142]:
char['level']

In [150]:
lvls = [x for x in dmg.keys() if x < tlvl]
lvls.sort()
target_dmg = dmg.get(lvls[-1])

2024-11-22 08:27:24,451 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.159:9042, scheduling retry in 600.0 seconds: [Errno None] Tried connecting to [('10.1.2.159', 9042)]. Last error: timed out


In [ ]:
for k,v in target_dmg.items():
    print(k,v)

Force [OrderedMapSerializedKey([(1, 10)])]


2024-11-22 08:27:50,831 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.166:9042, scheduling retry in 600.0 seconds: [Errno None] Tried connecting to [('10.1.2.166', 9042)]. Last error: timed out
2024-11-22 08:27:57,343 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.166:9042, scheduling retry in 528.0 seconds: [Errno None] Tried connecting to [('10.1.2.166', 9042)]. Last error: timed out
2024-11-22 08:29:04,476 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.166:9042, scheduling retry in 600.0 seconds: [Errno None] Tried connecting to [('10.1.2.166', 9042)]. Last error: timed out
2024-11-22 08:29:14,552 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.159:9042, scheduling retry in 552.0 seconds: [Errno None] Tried connecting to [('10.1.2.159', 9042)]. Last error: timed out
2024-11-22 08:30:05,498 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.166:9042, scheduling retry in 576.0 sec

In [163]:
import random

2024-11-22 08:34:30,663 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.159:9042, scheduling retry in 600.0 seconds: [Errno None] Tried connecting to [('10.1.2.159', 9042)]. Last error: timed out


In [ ]:
tlvl=6
lvls = [x for x in dmg.keys() if x < tlvl]
lvls.sort()
target_dmg = dmg.get(lvls[-1])
results = {}
for k,v in dict(target_dmg).items():
    results[k] = []
    for l,w in v[0].items():
        for i in range(l):
            # print(random.randint(1,w))
            results[k].append(random.randint(1,w))
results

{'Force': [9]}

2024-11-22 08:36:50,347 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.166:9042, scheduling retry in 528.0 seconds: [Errno None] Tried connecting to [('10.1.2.166', 9042)]. Last error: timed out
2024-11-22 08:36:53,395 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.166:9042, scheduling retry in 594.0 seconds: [Errno None] Tried connecting to [('10.1.2.166', 9042)]. Last error: timed out
2024-11-22 08:37:29,563 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.159:9042, scheduling retry in 600.0 seconds: [Errno None] Tried connecting to [('10.1.2.159', 9042)]. Last error: timed out
2024-11-22 08:37:55,930 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.166:9042, scheduling retry in 600.0 seconds: [Errno None] Tried connecting to [('10.1.2.166', 9042)]. Last error: timed out
2024-11-22 08:38:31,629 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.159:9042, scheduling retry in 600.0 sec

In [158]:
char

{'campaign_id': 'abcdef',
 'user_id': 'chorky#8402',
 'ac': 18,
 'align': None,
 'background': None,
 'char_name': 'Bahlok',
 'chr': 20,
 'field_7_': None,
 'con': None,
 'curr_hp': None,
 'dex': None,
 'hit_dice': None,
 'int': None,
 'level': None,
 'max_hp': None,
 'prof_bonus': None,
 'str': None,
 'wis': None}

In [175]:
spell

[{'id': 'ea42031d-4ab2-4d4a-bbb8-f6c0c80642bf',
  'addspellmod': True,
  'classes': ['Warlock'],
  'components': 'V, S',
  'conc': False,
  'dmg': OrderedMapSerializedKey([(1, OrderedMapSerializedKey([('Force', [OrderedMapSerializedKey([(1, 10)])])]))]),
  'duration': 'Instantaneous',
  'hard_mod': 0,
  'lvl': 0,
  'mult': 1,
  'name': 'Eldritch Blast',
  'range': '120 feet',
  'save': 'nan',
  'time': '1 action',
  'upcast': OrderedMapSerializedKey([])}]

2024-11-22 08:46:06,975 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.159:9042, scheduling retry in 600.0 seconds: [Errno None] Tried connecting to [('10.1.2.159', 9042)]. Last error: timed out


In [ ]:
## for converting spell damage and upcasts back into a Python dictionary
for row in spell:
    dmg_dict = {}
    dmg = row['dmg']
    for level, dmgmap in dmg.items():
        dmg_dict[level] = {}
        for dmgtype,die_list in dmgmap.items():
            dmg_dict[level][dmgtype] = []
            for die in die_list:
                dmg_dict[level][dmgtype].append(dict(die))
    upcast = dict(row['upcast'])
    print(dmg_dict,upcast)



{1: {'Cold': [{4: 6}]}} {}


2024-11-22 11:25:56,164 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.166:9042, scheduling retry in 510.0 seconds: [Errno None] Tried connecting to [('10.1.2.166', 9042)]. Last error: timed out
2024-11-22 11:27:20,022 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.159:9042, scheduling retry in 540.0 seconds: [Errno None] Tried connecting to [('10.1.2.159', 9042)]. Last error: timed out
2024-11-22 11:28:01,671 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.159:9042, scheduling retry in 570.0 seconds: [Errno None] Tried connecting to [('10.1.2.159', 9042)]. Last error: timed out
2024-11-22 11:28:07,054 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.166:9042, scheduling retry in 600.0 seconds: [Errno None] Tried connecting to [('10.1.2.166', 9042)]. Last error: timed out
2024-11-22 11:28:59,964 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.166:9042, scheduling retry in 600.0 sec

In [187]:
spell = db.exec_query(
    "SELECT * FROM spells WHERE name='Hunger of Hadar' ALLOW FILTERING;"
)['rows']

2024-11-22 08:54:57,622 - __main__ - INFO -   [x] DB on_request: query succeeded: SELECT * FROM spells WHERE name='Hunger of Hadar' ALLOW FILTERING;


In [188]:
spell

[{'id': '0c1c892e-5e9a-42d3-b950-0a2444b0ca7f',
  'addspellmod': True,
  'classes': ['Warlock'],
  'components': 'V, S, M (a pickled octopus tentacle)',
  'conc': True,
  'dmg': OrderedMapSerializedKey([(1, OrderedMapSerializedKey([('Acid', [OrderedMapSerializedKey([(2, 6)])]), ('Cold', [OrderedMapSerializedKey([(2, 6)])])]))]),
  'duration': 'Concentration, up to 1 minute',
  'hard_mod': 0,
  'lvl': 3,
  'mult': 1,
  'name': 'Hunger of Hadar',
  'range': '150 feet',
  'save': 'DEX',
  'time': '1 action',
  'upcast': OrderedMapSerializedKey([])}]

In [189]:
for row in spell:
    dmg_dict = {}
    dmg = row['dmg']
    for level, dmgmap in dmg.items():
        dmg_dict[level] = {}
        for dmgtype,die_list in dmgmap.items():
            dmg_dict[level][dmgtype] = []
            for die in die_list:
                dmg_dict[level][dmgtype].append(dict(die))
    upcast = dict(row['upcast'])
    print(dmg_dict,upcast)

{1: {'Acid': [{2: 6}], 'Cold': [{2: 6}]}} {}


In [235]:
spell = db.exec_query(
    "SELECT * FROM spells WHERE name='Investiture of Ice' ALLOW FILTERING;"
)['rows']
spell

2024-11-22 10:19:39,140 - __main__ - INFO -   [x] DB on_request: query succeeded: SELECT * FROM spells WHERE name='Investiture of Ice' ALLOW FILTERING;


[{'id': 'ea9941fd-917e-4a85-9079-d286a524d58c',
  'addspellmod': True,
  'classes': ['Druid', 'Sorcerer', 'Warlock', 'Wizard'],
  'components': 'V, S',
  'conc': True,
  'dmg': OrderedMapSerializedKey([(1, OrderedMapSerializedKey([('Cold', [OrderedMapSerializedKey([(4, 6)])])]))]),
  'duration': 'Concentration, up to 10 minutes',
  'hard_mod': 0,
  'lvl': 6,
  'mult': 1,
  'name': 'Investiture of Ice',
  'range': 'Self',
  'save': 'CON',
  'time': '1 action',
  'upcast': OrderedMapSerializedKey([])}]

In [ ]:
#need handling for 
# 
#   when the casting slot level is less than the spell's level - shouldn't be able to cast
#   when the spell is not in the character's known spells list
#   when there's a save required - include the required save type
#   return range and casting time, too?

#get these from the spell query
s=spell[0]
mult=s['mult']
hard_mod = s['hard_mod']
upcast = dict(s['upcast'])
dmg = s['dmg']
#get this from the character 
tlvl=6
#get this from the entered command
slot_lvl = 6


## determine any additional upcast dice
if len(upcast.keys())>0:
    upcast_lvl = list(upcast.keys())[0]
else:
    upcast_lvl = 21
if upcast_lvl <= slot_lvl:
    upcast_dice = slot_lvl - upcast_lvl + 1
else:
    upcast_dice = 0
print("upcast dice: {}".format(upcast_dice))

#find the char level that applies
lvls = [x for x in dmg.keys() if x <= tlvl]
lvls.sort()

#get the applicable damage at that char level
dmg = spell[0]['dmg']
target_dmg = dmg.get(lvls[-1])

#build results for die rolls (base and modified)
results = {}
mod_results = {}
#iterate through all applicable damage types
for k,v in dict(target_dmg).items():
    #populate a list of die rolls
    results[k] = []
    mod_results[k] = []
    for l,w in v[0].items():
        #number of upcast dice plus the number of rolls times how many beams/casts you get
        for i in range(upcast_dice + (l*mult)):
            # print(random.randint(1,w))
            roll = random.randint(1,w)
            results[k].append(roll)
            mod_results[k].append(roll+hard_mod)
# display(
#     results,
#     mod_results
# )
display(
    {
        'results':results,
        'modified_results':mod_results,
        'save':s['save']
    }
)

upcast dice: 0


{'results': {'Cold': [6, 5, 6, 4]},
 'modified_results': {'Cold': [6, 5, 6, 4]},
 'save': 'CON'}

2024-11-22 10:21:55,156 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.166:9042, scheduling retry in 600.0 seconds: [Errno None] Tried connecting to [('10.1.2.166', 9042)]. Last error: timed out
2024-11-22 10:22:01,144 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.166:9042, scheduling retry in 600.0 seconds: [Errno None] Tried connecting to [('10.1.2.166', 9042)]. Last error: timed out
2024-11-22 10:22:30,426 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.166:9042, scheduling retry in 540.0 seconds: [Errno None] Tried connecting to [('10.1.2.166', 9042)]. Last error: timed out
2024-11-22 10:22:46,096 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.166:9042, scheduling retry in 600.0 seconds: [Errno None] Tried connecting to [('10.1.2.166', 9042)]. Last error: timed out
2024-11-22 10:22:52,734 - cassandra.pool - WARNING - Error attempting to reconnect to 10.1.2.159:9042, scheduling retry in 588.0 sec